In [1]:
import pandas as pd

In [3]:
df = pd.read_csv("../data/titanic.csv")
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
df.drop(['PassengerId','Name','SibSp','Parch','Ticket','Cabin','Embarked'],axis='columns',inplace=True)
df.head()

,Survived,Pclass,Sex,Age,Fare
0,0,3,male,22.0,7.2500
1,1,1,female,38.0,71.2833
2,1,3,female,26.0,7.9250
3,1,1,female,35.0,53.1000
4,0,3,male,35.0,8.0500


In [5]:
inputs = df.drop('Survived',axis='columns')
target = df.Survived

In [6]:
dummies = pd.get_dummies(inputs.Sex)
dummies.head(3)

,female,male
0,False,True
1,True,False
2,True,False


In [7]:
inputs = pd.concat([inputs,dummies],axis='columns')
inputs.head(3)

,Pclass,Sex,Age,Fare,female,male
0,3,male,22.0,7.2500,False,True
1,1,female,38.0,71.2833,True,False
2,3,female,26.0,7.9250,True,False


I am dropping male column as well because of dummy variable trap theory. One column is enough to repressent male vs female

In [8]:
inputs.drop(['Sex','male'],axis='columns',inplace=True)
inputs.head(3)

,Pclass,Age,Fare,female
0,3,22.0,7.2500,False
1,1,38.0,71.2833,True
2,3,26.0,7.9250,True


In [9]:
inputs.columns[inputs.isna().any()]

Index(['Age'], dtype='object')

In [10]:
inputs.Age[:10]

0    22.0
1    38.0
2    26.0
3    35.0
4    35.0
5     NaN
6    54.0
7     2.0
8    27.0
9    14.0
Name: Age, dtype: float64

In [11]:
# popular approach is to take a mean of the entire column and fill the NA values
inputs.Age = inputs.Age.fillna(inputs.Age.mean())
inputs.head()

,Pclass,Age,Fare,female
0,3,22.0,7.2500,False
1,1,38.0,71.2833,True
2,3,26.0,7.9250,True
3,1,35.0,53.1000,True
4,3,35.0,8.0500,False


In [12]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(inputs,target,test_size=0.3)

In [13]:
from sklearn.naive_bayes import GaussianNB
model = GaussianNB()

In [14]:
model.fit(X_train,y_train)

GaussianNB()

In [15]:
model.score(X_test,y_test)

0.7873134328358209

In [16]:
X_test[0:10]

,Pclass,Age,Fare,female
134,2,25.00,13.0000,False
236,2,44.00,26.0000,False
499,3,24.00,7.7958,False
443,2,28.00,13.0000,True
852,3,9.00,15.2458,True
755,2,0.67,14.5000,False
780,3,13.00,7.2292,True
112,3,22.00,8.0500,False
758,3,34.00,8.0500,False
535,2,7.00,26.2500,True


In [17]:
y_test[0:10]

134    0
236    0
499    0
443    1
852    0
755    1
780    1
112    0
758    0
535    1
Name: Survived, dtype: int64

In [18]:
model.predict(X_test[0:10])

array([0, 0, 0, 1, 1, 0, 1, 0, 0, 1])

In [19]:
model.predict_proba(X_test[:10])

array([[0.91395278, 0.08604722],
       [0.90595659, 0.09404341],
       [0.95811554, 0.04188446],
       [0.21731962, 0.78268038],
       [0.28809215, 0.71190785],
       [0.84329747, 0.15670253],
       [0.31718104, 0.68281896],
       [0.95694038, 0.04305962],
       [0.96065232, 0.03934768],
       [0.13917537, 0.86082463]])

Calculate the score using cross validation

In [20]:
from sklearn.model_selection import cross_val_score
cross_val_score(GaussianNB(),X_train, y_train, cv=5)

array([0.72      , 0.776     , 0.816     , 0.79032258, 0.74193548])